In [1]:
import pandas as pd
import numpy as np
from sklearn import linear_model
import math
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.models import load_model
from keras.layers import LSTM, Dense, Dropout
from sklearn.preprocessing import MinMaxScaler
from keras.losses import CategoricalCrossentropy
from keras.callbacks import EarlyStopping
import tensorflow


In [2]:
#Read data from csv
df_orig = pd.read_csv("casos_all_likereal_noCTEorN3.txt", error_bad_lines=False)
#df = df.apply(pd.to_numeric)
df_orig

C:\Users\migue\AppData\Local\Temp\ipykernel_16780\2685560779.py:2: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df_orig = pd.read_csv("casos_all_likereal_noCTEorN3.txt", error_bad_lines=False)


,tamano,tiempo,tamano.1,tiempo.1,tamano.2,tiempo.2,tipo
0,27901,0.004095,139509,0.018953,279018,0.030094,1
1,24040,0.004035,120200,0.004679,240400,0.029736,1
2,27442,0.025550,137213,0.023659,274427,0.005009,1
3,27600,0.006135,138000,0.002367,276000,0.018792,1
4,23048,0.005023,115244,0.004662,230488,0.029635,1
...,...,...,...,...,...,...,...
783995,1236,0.003636,6182,0.101658,12365,0.128430,4
783996,1383,0.003481,6915,0.120499,13830,0.404301,4
783997,1257,0.000442,6285,0.038711,12571,0.458160,4
783998,1538,0.001987,7690,0.157302,15380,0.059001,4


In [3]:
#Split dataframe to make smaller in order to run quicker tests before final model
df = df_orig#.iloc[:312500, :]
df

,tamano,tiempo,tamano.1,tiempo.1,tamano.2,tiempo.2,tipo
0,27901,0.004095,139509,0.018953,279018,0.030094,1
1,24040,0.004035,120200,0.004679,240400,0.029736,1
2,27442,0.025550,137213,0.023659,274427,0.005009,1
3,27600,0.006135,138000,0.002367,276000,0.018792,1
4,23048,0.005023,115244,0.004662,230488,0.029635,1
...,...,...,...,...,...,...,...
783995,1236,0.003636,6182,0.101658,12365,0.128430,4
783996,1383,0.003481,6915,0.120499,13830,0.404301,4
783997,1257,0.000442,6285,0.038711,12571,0.458160,4
783998,1538,0.001987,7690,0.157302,15380,0.059001,4


In [4]:
for i in range(0, len(df)-1):
  if df.tipo.values[i] < 1 or df.tipo.values[i] > 4:
    df = df.drop(i, axis=0)
df = df.dropna()

df

,tamano,tiempo,tamano.1,tiempo.1,tamano.2,tiempo.2,tipo
0,27901,0.004095,139509,0.018953,279018,0.030094,1
1,24040,0.004035,120200,0.004679,240400,0.029736,1
2,27442,0.025550,137213,0.023659,274427,0.005009,1
3,27600,0.006135,138000,0.002367,276000,0.018792,1
4,23048,0.005023,115244,0.004662,230488,0.029635,1
...,...,...,...,...,...,...,...
783995,1236,0.003636,6182,0.101658,12365,0.128430,4
783996,1383,0.003481,6915,0.120499,13830,0.404301,4
783997,1257,0.000442,6285,0.038711,12571,0.458160,4
783998,1538,0.001987,7690,0.157302,15380,0.059001,4


In [5]:
#Scale tamano and tiempo columns so that their values are between [0, 1]
#scaler = MinMaxScaler(feature_range=(0,1))
#df[["tamano", "tiempo","tamano.1", "tiempo.1","tamano.2", "tiempo.2"]] = scaler.fit_transform(df[["tamano", "tiempo","tamano.1", "tiempo.1","tamano.2", "tiempo.2"]])

In [6]:
#delete size data to test predictions
#df = df.drop(['tamano', 'tamano.1', 'tamano.2'], axis=1)
#df

In [7]:
#def is_float(value):
#  try:
#    float(value)
#    return True
#  except:
#    return False

#del_ids = {}
#for j in range(0, 7):
#  col = df.iloc[:,j]
#  col = np.array(col)
#  for i in range(0,len(col)-1):
#    if is_float(col[i]) == False and i in del_ids == False:
#      df[i][j] = float(df[i][j]);

#for id in del_ids:
#  df.drop(id, axis=0);

#len(del_ids)


In [8]:
#drop type column to obtain the x_train data use 6 if size included 3 if not
x_train = df.iloc[:, 0:6]
#x_train = df.iloc[:, 0:3]

#y_train = df.iloc[:,6]
x_train

,tamano,tiempo,tamano.1,tiempo.1,tamano.2,tiempo.2
0,27901,0.004095,139509,0.018953,279018,0.030094
1,24040,0.004035,120200,0.004679,240400,0.029736
2,27442,0.025550,137213,0.023659,274427,0.005009
3,27600,0.006135,138000,0.002367,276000,0.018792
4,23048,0.005023,115244,0.004662,230488,0.029635
...,...,...,...,...,...,...
783995,1236,0.003636,6182,0.101658,12365,0.128430
783996,1383,0.003481,6915,0.120499,13830,0.404301
783997,1257,0.000442,6285,0.038711,12571,0.458160
783998,1538,0.001987,7690,0.157302,15380,0.059001


In [9]:
y_train = []
for i in range(0,len(df)):
  high = df.tipo.values[i]
  if high == 1:
    y_train.append([1,0,0,0])
  elif high == 2:
    y_train.append([0,1,0,0])
  elif high == 3:
    y_train.append([0,0,1,0])
  elif high == 4:
    y_train.append([0,0,0,1])
  else:
#    x_train.delete(i)
    print(str(i) + " " + str(df.tipo.values[i]))

len(y_train)

784000

In [10]:
#reshape data to the type the AI model needs
x_train, y_train = np.array(x_train), np.array(y_train)
x_train = np.asarray(x_train).astype('float32')
y_train = np.asarray(y_train).astype('float32')

In [11]:
#Split the data to allow train and test the model
#stratify te hace que haya una proporcion igual de cada categoria en ambos sets a ver klkpasa
x_trn, x_test, y_trn, y_test = train_test_split(x_train, y_train, stratify=y_train)

In [12]:
len(x_trn)

588000

Note, the most confusing thing here is that the shape of the input to the model is defined as an argument on the first hidden layer. This means that the line of code that adds the first layer is doing 2 things, defining the input or visible layer and the first hidden layer.

In [13]:
#model = load_model("model_4pcntError.h5")
#model.summary()

In [14]:
#Use the model to predict the test data in order to find the success percent in the test data

#x_test = np.array(x_test)
#y_test = np.array(y_test)

predictions = model.predict(x_test)
predictions

NameError: name 'model' is not defined

In [ ]:
#Calculate error generated in the predictions of test data
errors = predictions - y_test
errors

array([[ 1.46065885e-02, -3.05476725e-01,  1.58256218e-01,
         1.32613823e-01],
       [ 8.80209580e-02, -7.17140317e-01,  4.99600083e-01,
         1.29519284e-01],
       [ 1.12284385e-02, -2.37518072e-01,  1.21350229e-01,
         1.04939446e-01],
       ...,
       [-4.76577699e-01,  1.34912640e-04,  4.33915317e-01,
         4.25274484e-02],
       [ 1.70433335e-02, -4.62872624e-01,  2.18534485e-01,
         2.27294847e-01],
       [ 6.57891436e-03, -1.69939101e-01,  7.87938684e-02,
         8.45663026e-02]], dtype=float32)

In [ ]:
real_preds = []
for i in range(0,len(predictions)):
  p = predictions[i]
  if max(p) == p[0]:
    real_preds.append([1,0,0,0])
  elif max(p) == p[1]:
    real_preds.append([0,1,0,0])
  elif max(p) == p[2]:
    real_preds.append([0,0,1,0])
  elif max(p) == p[3]:
    real_preds.append([0,0,0,1])
real_preds

[[0, 1, 0, 0],
 [0, 0, 1, 0],
 [0, 1, 0, 0],
 [1, 0, 0, 0],
 [0, 0, 0, 1],
 [0, 0, 0, 1],
 [0, 0, 1, 0],
 [0, 0, 1, 0],
 [0, 1, 0, 0],
 [0, 0, 1, 0],
 [1, 0, 0, 0],
 [1, 0, 0, 0],
 [0, 0, 1, 0],
 [0, 0, 1, 0],
 [0, 0, 0, 1],
 [0, 1, 0, 0],
 [0, 1, 0, 0],
 [0, 0, 1, 0],
 [0, 1, 0, 0],
 [1, 0, 0, 0],
 [0, 1, 0, 0],
 [0, 1, 0, 0],
 [1, 0, 0, 0],
 [0, 1, 0, 0],
 [0, 0, 0, 1],
 [0, 0, 1, 0],
 [1, 0, 0, 0],
 [0, 0, 0, 1],
 [0, 0, 1, 0],
 [0, 1, 0, 0],
 [0, 1, 0, 0],
 [0, 1, 0, 0],
 [0, 0, 1, 0],
 [0, 1, 0, 0],
 [0, 0, 1, 0],
 [0, 0, 0, 1],
 [1, 0, 0, 0],
 [0, 1, 0, 0],
 [1, 0, 0, 0],
 [0, 0, 1, 0],
 [0, 1, 0, 0],
 [0, 1, 0, 0],
 [0, 0, 0, 1],
 [0, 0, 0, 1],
 [0, 0, 1, 0],
 [0, 0, 0, 1],
 [0, 0, 1, 0],
 [0, 0, 1, 0],
 [0, 1, 0, 0],
 [0, 0, 0, 1],
 [0, 0, 0, 1],
 [0, 1, 0, 0],
 [0, 0, 0, 1],
 [0, 1, 0, 0],
 [0, 0, 0, 1],
 [0, 0, 1, 0],
 [0, 0, 1, 0],
 [0, 0, 0, 1],
 [0, 1, 0, 0],
 [0, 1, 0, 0],
 [0, 0, 0, 1],
 [0, 0, 0, 1],
 [0, 1, 0, 0],
 [0, 1, 0, 0],
 [0, 1, 0, 0],
 [0, 0, 1, 0],
 [0, 1, 0,

In [ ]:
#Calculate error generated in the "real"predictions of test data
errors = real_preds - y_test
errors

array([[ 0.,  0.,  0.,  0.],
       [ 0., -1.,  1.,  0.],
       [ 0.,  0.,  0.,  0.],
       ...,
       [ 0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.]])

In [ ]:
error_count = 0
err = real_preds - y_test
for i in range(0, len(err)):
  if max(err[i]) != min(err[i]):
    error_count += 1
print(str(error_count) + " / " + str(len(real_preds)) + " => real error% of " + str(error_count/len(real_preds)*100))

54564 / 196000 => real error% of 27.838775510204083


Cappear el tiempoEjecucion maximo con double.max o similar para poder meter tamaños mas grandes al calcula tiempo con n2 y n3 (cuidado introduce mas ruido!!) **done**

Añadir otra error count aceptando como aciertos mas casos: EJ: un n2 lo llama n3 es un acierto. **idk**

Probar a sustituir los tamaños por los porcentajes de tam maximo
o probar a quitar los tamaños
=> para verificar que no aprende a predecir solo en base al tamaño **done**


Probar con casos reales del programa de alvaro. **todo**


t2 - t1 y t3 - t1 => plot

Predecir basandose en un documento con tiempos externo a los de train y test


In [18]:
real_df = pd.read_csv("new_tiempos_reales.txt", error_bad_lines=False)
real_df

C:\Users\migue\AppData\Local\Temp\ipykernel_16780\3101620717.py:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  real_df = pd.read_csv("new_tiempos_reales.txt", error_bad_lines=False)


,N,seconds,N.1,seconds.1,N.2,seconds.2,tipo
0,21474,0.018019,107374,0.047188,212600,0.095220,3
1,10000,0.017800,50000,0.075275,100000,0.187217,3
2,1000,0.006315,50000,0.013816,99000,0.030289,2
3,10,0.001838,100000,0.033445,200000,0.065974,2
4,10000,0.004839,50000,0.010540,100000,0.016269,2
5,20000,0.009522,500000,0.059982,990000,0.060529,2
6,20,0.012815,500,0.020178,1000,0.040146,4
7,10,0.004479,50,0.018389,100,0.029830,4
8,2147,0.005759,10735,0.078523,21255,0.313791,4
9,100,0.022623,500,0.034553,990,0.066085,4


In [19]:
#real_df = real_df.drop(['N', 'N.1', 'N.2'], axis=1)
x_train_r = real_df.iloc[:, 0:6]
#x_train = real_df.iloc[:, 0:3]

x_train_r = np.array(x_train_r)
x_train_r

array([[2.14740e+04, 1.80189e-02, 1.07374e+05, 4.71880e-02, 2.12600e+05,
        9.52201e-02],
       [1.00000e+04, 1.77996e-02, 5.00000e+04, 7.52747e-02, 1.00000e+05,
        1.87217e-01],
       [1.00000e+03, 6.31468e-03, 5.00000e+04, 1.38162e-02, 9.90000e+04,
        3.02892e-02],
       [1.00000e+01, 1.83829e-03, 1.00000e+05, 3.34450e-02, 2.00000e+05,
        6.59738e-02],
       [1.00000e+04, 4.83884e-03, 5.00000e+04, 1.05405e-02, 1.00000e+05,
        1.62693e-02],
       [2.00000e+04, 9.52196e-03, 5.00000e+05, 5.99822e-02, 9.90000e+05,
        6.05295e-02],
       [2.00000e+01, 1.28153e-02, 5.00000e+02, 2.01785e-02, 1.00000e+03,
        4.01461e-02],
       [1.00000e+01, 4.47880e-03, 5.00000e+01, 1.83888e-02, 1.00000e+02,
        2.98302e-02],
       [2.14700e+03, 5.75876e-03, 1.07350e+04, 7.85231e-02, 2.12550e+04,
        3.13791e-01],
       [1.00000e+02, 2.26230e-02, 5.00000e+02, 3.45530e-02, 9.90000e+02,
        6.60845e-02],
       [9.00000e+04, 4.79819e-04, 4.90000e+05, 5.1

In [20]:
y_train_r = []
for i in range(0,len(real_df)):
  high = real_df.tipo.values[i]
  if high == 1:
    y_train_r.append([1,0,0,0])
  elif high == 2:
    y_train_r.append([0,1,0,0])
  elif high == 3:
    y_train_r.append([0,0,1,0])
  elif high == 4:
    y_train_r.append([0,0,0,1])
  else:
    print(str(i) + " " + str(real_df.tipo.values[i]))

len(y_train_r)

13

In [21]:
y_train_r = np.array(y_train_r)
y_train_r

array([[0, 0, 1, 0],
       [0, 0, 1, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 0, 0, 1],
       [0, 0, 0, 1],
       [0, 0, 0, 1],
       [0, 0, 0, 1],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0]])

In [24]:
#Generate a sequential model with 4 layers 
#The last one has 6 nodes, one for each execution time order [logn, n, nlogn, n2]
#The ones in between have arbitrary values obtain through testing ( still may be swapped )

#redes GRU igual esta interesante

model = Sequential()
#model.add(no. de entradas de cada caso) 
#esto es asi, en la libreria keras, en el modelo secuencial, se define en input_shape el nº de nodos de la entrada
model.add(LSTM(6, return_sequences=True, input_shape=(x_train.shape[1], 1)))

model.add(Dense(6))
model.add(LSTM(6, return_sequences=False))

model.add(Dense(4, activation = 'softmax'))

#model.add(Dense(50, activation='relu'))
#model.add(Dense(50, activation='relu'))
#model.add(Dense(25, activation='relu'))
#model.add(Dense(5, activation='relu'))

#Compile the model previously created with the chosen parameters
#optimizer='sgd', loss = 'mse'
model.compile(optimizer='adam', loss = CategoricalCrossentropy(), metrics=['accuracy'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_2 (LSTM)               (None, 6)                 192       
                                                                 
 dense_4 (Dense)             (None, 4)                 28        
                                                                 
Total params: 220
Trainable params: 220
Non-trainable params: 0
_________________________________________________________________


In [25]:
#Train the model with the 'train' portion of the data 
#The other parameters are tuned down in order to reduce train time but should be higher
callback = EarlyStopping(monitor='loss', patience = 2, restore_best_weights=True)

model.fit(x_trn, y_trn, epochs=4, callbacks=[callback], validation_data=(x_train_r, y_train_r))

Epoch 1/4
18375/18375 [==============================] - 36s 2ms/step - loss: 0.9436 - accuracy: 0.5907 - val_loss: 4.5819 - val_accuracy: 0.3077
Epoch 2/4
18375/18375 [==============================] - 36s 2ms/step - loss: 0.6507 - accuracy: 0.7205 - val_loss: 5.3530 - val_accuracy: 0.3077
Epoch 3/4
18375/18375 [==============================] - 34s 2ms/step - loss: 0.6251 - accuracy: 0.7323 - val_loss: 5.4847 - val_accuracy: 0.3077
Epoch 4/4
18375/18375 [==============================] - 34s 2ms/step - loss: 0.6204 - accuracy: 0.7347 - val_loss: 5.6209 - val_accuracy: 0.2308


In [26]:
#Use this to train more a trained model :)
#callback = EarlyStopping(monitor='loss', patience = 2, restore_best_weights=True)

#model.fit(x_trn, y_trn, batch_size=128, epochs=4, callbacks=[callback])

In [27]:
#Use this to train more a trained model :)
#callback = EarlyStopping(monitor='loss', patience = 2, restore_best_weights=True)

#model.fit(x_trn, y_trn, batch_size=128, epochs=8, callbacks=[callback])

In [28]:
#Save trained model to a file
model.save("model_noCTEorN3.h5")

In [29]:
real_df_preds = model.predict(x_train_r)
real_df_preds

array([[3.60434681e-01, 3.13758574e-05, 6.38209641e-01, 1.32437295e-03],
       [2.89658427e-01, 2.05551191e-06, 7.06973970e-01, 3.36549827e-03],
       [2.31057525e-01, 1.98380426e-01, 5.54511607e-01, 1.60504747e-02],
       [9.99344289e-01, 1.81795131e-05, 6.32327865e-04, 5.18363686e-06],
       [1.35891587e-01, 4.51638758e-01, 3.89043182e-01, 2.34264694e-02],
       [2.44932070e-01, 5.64794522e-04, 7.44909823e-01, 9.59328748e-03],
       [9.99346554e-01, 2.07501034e-05, 6.27494068e-04, 5.24089182e-06],
       [9.99571502e-01, 1.92620901e-05, 4.06941050e-04, 2.30225783e-06],
       [8.05901084e-03, 5.11539110e-05, 5.48146009e-01, 4.43743825e-01],
       [9.99341309e-01, 1.80634452e-05, 6.35416538e-04, 5.19884998e-06],
       [1.13760317e-02, 8.29291821e-01, 8.88423994e-02, 7.04897344e-02],
       [1.20357815e-02, 8.26325297e-01, 9.20088589e-02, 6.96301013e-02],
       [1.16510717e-02, 8.25230718e-01, 9.09737796e-02, 7.21443668e-02]],
      dtype=float32)

In [30]:
err = real_df_preds - y_train_r
err

array([[ 3.60434681e-01,  3.13758574e-05, -3.61790359e-01,
         1.32437295e-03],
       [ 2.89658427e-01,  2.05551191e-06, -2.93026030e-01,
         3.36549827e-03],
       [ 2.31057525e-01, -8.01619574e-01,  5.54511607e-01,
         1.60504747e-02],
       [ 9.99344289e-01, -9.99981820e-01,  6.32327865e-04,
         5.18363686e-06],
       [ 1.35891587e-01, -5.48361242e-01,  3.89043182e-01,
         2.34264694e-02],
       [ 2.44932070e-01, -9.99435205e-01,  7.44909823e-01,
         9.59328748e-03],
       [ 9.99346554e-01,  2.07501034e-05,  6.27494068e-04,
        -9.99994759e-01],
       [ 9.99571502e-01,  1.92620901e-05,  4.06941050e-04,
        -9.99997698e-01],
       [ 8.05901084e-03,  5.11539110e-05,  5.48146009e-01,
        -5.56256175e-01],
       [ 9.99341309e-01,  1.80634452e-05,  6.35416538e-04,
        -9.99994801e-01],
       [-9.88623968e-01,  8.29291821e-01,  8.88423994e-02,
         7.04897344e-02],
       [-9.87964218e-01,  8.26325297e-01,  9.20088589e-02,
      

In [31]:
real_preds = []
for i in range(0,len(real_df_preds)):
  p = real_df_preds[i]
  if max(p) == p[0]:
    real_preds.append([1,0,0,0])
  elif max(p) == p[1]:
    real_preds.append([0,1,0,0])
  elif max(p) == p[2]:
    real_preds.append([0,0,1,0])
  elif max(p) == p[3]:
    real_preds.append([0,0,0,1])

real_preds

[[0, 0, 1, 0],
 [0, 0, 1, 0],
 [0, 0, 1, 0],
 [1, 0, 0, 0],
 [0, 1, 0, 0],
 [0, 0, 1, 0],
 [1, 0, 0, 0],
 [1, 0, 0, 0],
 [0, 0, 1, 0],
 [1, 0, 0, 0],
 [0, 1, 0, 0],
 [0, 1, 0, 0],
 [0, 1, 0, 0]]

In [32]:
err = real_preds - y_train_r
err

array([[ 0,  0,  0,  0],
       [ 0,  0,  0,  0],
       [ 0, -1,  1,  0],
       [ 1, -1,  0,  0],
       [ 0,  0,  0,  0],
       [ 0, -1,  1,  0],
       [ 1,  0,  0, -1],
       [ 1,  0,  0, -1],
       [ 0,  0,  1, -1],
       [ 1,  0,  0, -1],
       [-1,  1,  0,  0],
       [-1,  1,  0,  0],
       [-1,  1,  0,  0]])

añadir O(1).
t2 - t1 y t3 - t1 => plot (esto no hace falta era para el debug)

In [33]:
error_count = 0
for i in range(0, len(err)):
  if max(err[i]) != min(err[i]):
    error_count += 1
print(str(error_count) + " / " + str(len(real_preds)) + " => real error% of " + str(error_count/len(real_preds)*100))

10 / 13 => real error% of 76.92307692307693


In [34]:
#error teniendo en cuenta la aplicacion real del programa ( no todos los fallos son fallos )
#ejemplo: espero un lineal y entregan un n2, lo interpreto como n3, el resultado seria "no es lineal" y no seria un fallo
#creo que este tipo de error_count solo se puede calcular aplicando el modelo a un problema donde espero un resultado y entregan otro, no se como hacerlo aqui

y_train = []#tipo de ejecucion real del programa
expected_train = []#tipo de ejecucion esperado por el problema
predictions_train = []#ordenes predecidos por la ia

correct = 0
partial_corrects = 0
errors = 0
for i in range(0, len(real_preds)):
  guess_id = np.argmax(predictions_train[i])
  answer_id = np.argmax(y_train[i])
  expected_id = np.argmax(expected_train[i])
  if guess_id == answer_id:
    correct += 1
  elif guess_id > answer_id and expected_id != answer_id:
    partial_corrects += 1
  else:
    errors += 1

print(str(errors) + " / " + str(len(real_preds)) + " => useful error% of " + str(errors/len(real_preds)*100))
print(str(correct) + " + " + str(partial_corrects) + " = " + str(correct+partial_corrects)  + " / " + str(len(real_preds)) + " => useful guesses% of " + str((correct+partial_corrects)/len(real_preds)*100))


#error_count = 0
#for i in range(0, len(real_preds)):
#  guess_id = np.argmax(real_preds[i])
#  answer_id = np.argmax(y_test[i])
#  if guess_id < answer_id:
#    error_count += 1
#print(str(error_count) + " / " + str(len(real_preds)) + " => useful error% of " + str(error_count/len(real_preds)*100))

IndexError: list index out of range